# Columbia Attention Token - Token Scoring System

## About This Notebook...

This notebook begins by importing the .csv file containing the cleaned Slack data from the 'slack_eda_and_wrangling' notebook. After creating a DataFrame from the file and reiewing it, the groupby() function is applied to get an idea of total counts that represent cumulative participation. Considering these counts, two columns are added, both constructed from simple equations that give a better breakdown of engagement. The first is the user's "reply ratio," which takes the sum of the replies to a user's posts and divides that by the number of user's responsible for those replies. The next is a "token score," which uses the reply ratio as one of its variables, along with the sum of the text length of each user's posts, the sum of reactions each user contributed, and the sum of attachments each user provided in posts, all of which have been given a unique weighting. The result is the each user's unique "token score," which is used for the distribution of coins minted in our smart contracts.

#### Imports and DataFrame Review

In [12]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import csv

In [13]:
# Import the csv file contained cleaned slack channel data
slack_csv_path = Path("./slack_cleaned.csv")
slack_df = pd.read_csv(slack_csv_path)
slack_df.head(20)

,channel_name,user,text,text_length,reply_count,reply_users_count,replies_true,day_name,day_type,time,dayparts,day_number,month,reactions_count,reactions_name,attachments,attachments_true,reaction_true
0,fintech,U023R27V74N,two upcoming conferences:\n• finovatefall sept...,415,0,0,False,Thursday,Weekday,1,Late Night,12,August,0,0,https://informaconnect.com/finovatefall/,True,True
1,fintech,U023R27V74N,also two free virtual expos sept 29-30:\n• blo...,168,0,0,False,Thursday,Weekday,1,Late Night,12,August,0,0,https://blockchain-expo.com/northamerica/,True,True
2,fintech,U023R27V74N,the columbia center of ai is having a symposiu...,170,0,0,False,Thursday,Weekday,18,Evening,30,September,1,muscle,https://www.eventbrite.com/e/cait-inaugural-sy...,True,True
3,fintech,U025DPVSGBT,"Y’all, free diy NFT class kicking off at 12pm ...",120,0,0,False,Sunday,Weekend,15,Afternoon,26,September,3,+1,https://buildspace.so/build-nfts,True,True
4,fintech,U024R294XHV,Anyone interested in next generation APIs here...,206,0,0,False,Wednesday,Weekday,12,Afternoon,6,October,2,fire,https://www.sigtech.com/platform/data,True,True
5,fintech,U025DPVSGBT,<https://medium.com/derivadex/what-are-perpetu...,208,0,0,False,Thursday,Weekday,14,Afternoon,7,October,1,orange_heart,https://medium.com/derivadex/what-are-perpetua...,True,True
6,fintech,U023R27V74N,<https://www.fastcompany.com/90669744/spotify-...,66,0,0,False,Monday,Weekday,16,Evening,30,August,0,0,https://www.fastcompany.com/90669744/spotify-t...,True,True
7,fintech,U023R27V74N,if you are interested in learning how azure su...,644,0,0,False,Monday,Weekday,22,Night,11,October,0,0,https://info.microsoft.com/ww-landing-use-ai-t...,True,True
8,fintech,U023R27V74N,Fintech Junction Summer Event\nThere is a free...,107,0,0,False,Tuesday,Weekday,16,Evening,22,June,2,+1,None,True,True
9,fintech,U023R27V74N,MoneyNext Open Banking Summit June 22-23\nIf y...,159,0,0,False,Friday,Weekday,19,Evening,18,June,6,+1,https://moneynext.tv/open-banking-summit/,True,True


#### Assessing Counts for Scoring

In [14]:
#print(slack_df.groupby(["user"])[['real_name']])
new_df = slack_df.groupby(["user"])[["text_length"]].sum()
print(new_df)
#by_month_df = slack_df.groupby(['user', 'real_name'])
#by_month_df.head(35)

             text_length
user                    
U023F2P1CKZ         1822
U023QT9923G         6485
U023R27V74N        24585
U023TR6SGJZ        12933
U024GNF18MB           73
U024GNX9CGM        12825
U024J4FLHHD         9278
U024J6725K8         1936
U024JN413AR          853
U024MNS46GM          443
U024QHZRR6E          387
U024R294XHV        11657
U024SCVTY5T          954
U024VFELUBZ         1014
U024VKV1V6K         1086
U024WBY5S5Q         3871
U024WMKUP0C          578
U024YNXFLUR          117
U024ZA73WKV          106
U02507X9M5G         1276
U02536C3K3P          601
U0253G8HJ0H          297
U0253QXQT2S         1746
U0253RQ4Q5U         1735
U02548Z2S69         2600
U0254U087B7         1711
U0255KNQ0N8          206
U0256T3ARB6           52
U0259QYTM1N          210
U025AJNRNN6         2344
U025AK2MCCA          432
U025DPVSGBT         5654
U025E9FJVNU         2195
U025G8XUG8L          217
U025GLTLJBE          627
U025JC6MEBY          507
U025M2H7U65            9
U025NEBV4C9          341


In [15]:
# Drop the columns that aren't needed
feat_slack_df = slack_df.drop(['text', 'day_name', 'day_type', 'time', 'dayparts', 'day_number', 'month', 'reaction_true'], axis=1)

feat_slack_df.head(20)

,channel_name,user,text_length,reply_count,reply_users_count,replies_true,reactions_count,reactions_name,attachments,attachments_true
0,fintech,U023R27V74N,415,0,0,False,0,0,https://informaconnect.com/finovatefall/,True
1,fintech,U023R27V74N,168,0,0,False,0,0,https://blockchain-expo.com/northamerica/,True
2,fintech,U023R27V74N,170,0,0,False,1,muscle,https://www.eventbrite.com/e/cait-inaugural-sy...,True
3,fintech,U025DPVSGBT,120,0,0,False,3,+1,https://buildspace.so/build-nfts,True
4,fintech,U024R294XHV,206,0,0,False,2,fire,https://www.sigtech.com/platform/data,True
5,fintech,U025DPVSGBT,208,0,0,False,1,orange_heart,https://medium.com/derivadex/what-are-perpetua...,True
6,fintech,U023R27V74N,66,0,0,False,0,0,https://www.fastcompany.com/90669744/spotify-t...,True
7,fintech,U023R27V74N,644,0,0,False,0,0,https://info.microsoft.com/ww-landing-use-ai-t...,True
8,fintech,U023R27V74N,107,0,0,False,2,+1,None,True
9,fintech,U023R27V74N,159,0,0,False,6,+1,https://moneynext.tv/open-banking-summit/,True


In [16]:
# Group the slack data by the User ID and create a new DataFrame from this action,
# then calculate the sum of the columns to get a count of the user's total interactions.

grouped_slack_df = feat_slack_df.groupby('user').sum()

grouped_slack_df

,text_length,reply_count,reply_users_count,replies_true,reactions_count,attachments_true
user,,,,,,
U023F2P1CKZ,1822,88,62,5,28,23
U023QT9923G,6485,2,2,2,10,66
U023R27V74N,24585,13,11,5,45,164
U023TR6SGJZ,12933,7,4,2,21,52
U024GNF18MB,73,0,0,0,1,2
U024GNX9CGM,12825,103,66,32,39,101
U024J4FLHHD,9278,3,3,2,18,64
U024J6725K8,1936,18,5,3,10,35
U024JN413AR,853,16,15,8,8,15


#### Remember:
- the 'text_length' column represents the total number of characters written in all posts by each user.
- the 'reply_count' column represents the total number of replies each user has gotten on their posts.
- the 'reply_users_count' column represents the total number of other users who replied to a user's posts.
- the 'attachments_true' column is a summed boolean representing the total number of times a post contained an attachment.
- the 'reactions_count' column represents the total number of reactions each user contributed.

#### Token Scoring System:
- each character the user has written is worth 2 points.
- each attachment a user has contributed is worth 20 points.
- the user's "reply ratio" is multiplied by 200 points.
- each reaction a user has contributed is worth 5 points.

In [17]:
# Check the data types of the columns

grouped_slack_df.dtypes

text_length          int64
reply_count          int64
reply_users_count    int64
replies_true         int64
reactions_count      int64
attachments_true     int64
dtype: object

### Creating New Columns for Reply Ratio and Token Score

In [18]:
# Create a reply ratio column
grouped_slack_df['reply_ratio'] = grouped_slack_df['reply_count'] / grouped_slack_df['reply_users_count']

# Review DataFrame for new column
grouped_slack_df.head(15)

,text_length,reply_count,reply_users_count,replies_true,reactions_count,attachments_true,reply_ratio
user,,,,,,,
U023F2P1CKZ,1822,88,62,5,28,23,1.419355
U023QT9923G,6485,2,2,2,10,66,1.000000
U023R27V74N,24585,13,11,5,45,164,1.181818
U023TR6SGJZ,12933,7,4,2,21,52,1.750000
U024GNF18MB,73,0,0,0,1,2,NaN
U024GNX9CGM,12825,103,66,32,39,101,1.560606
U024J4FLHHD,9278,3,3,2,18,64,1.000000
U024J6725K8,1936,18,5,3,10,35,3.600000
U024JN413AR,853,16,15,8,8,15,1.066667


In [19]:
# Drop NaN rows in the reply_ratio column
grouped_slack_df['reply_ratio'] = grouped_slack_df['reply_ratio'].fillna(0)

In [20]:
# Use the grouped data to write the conditional statements that will construct the point system for the eventual token distribution.

grouped_slack_df['token_score'] = ((2 * grouped_slack_df['text_length']) + (20 * grouped_slack_df['attachments_true'])
                                   + (200 * grouped_slack_df['reply_ratio']) + (5 * grouped_slack_df['reactions_count']))

# Review DataFrame for new column
grouped_slack_df

,text_length,reply_count,reply_users_count,replies_true,reactions_count,attachments_true,reply_ratio,token_score
user,,,,,,,,
U023F2P1CKZ,1822,88,62,5,28,23,1.419355,4527.870968
U023QT9923G,6485,2,2,2,10,66,1.000000,14540.000000
U023R27V74N,24585,13,11,5,45,164,1.181818,52911.363636
U023TR6SGJZ,12933,7,4,2,21,52,1.750000,27361.000000
U024GNF18MB,73,0,0,0,1,2,0.000000,191.000000
U024GNX9CGM,12825,103,66,32,39,101,1.560606,28177.121212
U024J4FLHHD,9278,3,3,2,18,64,1.000000,20126.000000
U024J6725K8,1936,18,5,3,10,35,3.600000,5342.000000
U024JN413AR,853,16,15,8,8,15,1.066667,2259.333333


In [21]:
# Save to csv
grouped_slack_df.to_csv('tss_slack.csv', index=True)